In [149]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score

: 

In [ ]:
df = pd.read_csv('/content/sample_data/Dataset.csv.csv', sep=';')
df


In [ ]:

df.info()


In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df = pd.read_csv('/content/sample_data/Dataset.csv.csv', sep=';', header=0)
print(df.columns)


In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
df

In [ ]:
df.info()

In [ ]:
df=df.sort_values(by=['id','date'])
df.head()

In [ ]:
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month
df.head()

In [ ]:
df.columns

: 

In [160]:
pollutants=[ 'O2', 'NO3', 'NO2', 'SO4', 'PO4', 'CL']

## Week 2 - 23 June 2025

In [ ]:
df=df.dropna(subset=pollutants)
df.head()

In [ ]:
df.isnull().sum()

In [163]:
# feature and target selection
X=df[['id','year']]
y=df[pollutants]

In [164]:
# encoding
X_encoded=pd.get_dummies(X,columns=['id'],drop_first=True)

In [165]:
# Train,test and split
X_train,X_test,y_train,y_test=train_test_split(
    X_encoded,y,test_size=0.2,random_state=42
)


In [ ]:
# train the model
model=MultiOutputRegressor(RandomForestRegressor(n_estimators=100,random_state=42))
model.fit(X_train,y_train)

In [167]:
# Evaluate the model
y_pred = model.predict(X_test)


In [ ]:
print("Model Performance on the Test Data:")
for i, pollutant in enumerate(pollutants):
    print(f'{pollutant}:')
    print('    MSE:', mean_squared_error(y_test.iloc[:, i], y_pred[:, i]))
    print('    R2:', r2_score(y_test.iloc[:, i], y_pred[:, i]))
    print()


In [ ]:
station_id='22'
year_input=2024
input_data=pd.DataFrame({'year':[year_input],'id':[station_id]})
input_encoded=pd.get_dummies(input_data,columns=['id'])

# align with training feature columns
missing_cols=set(X_encoded.columns)-set(input_encoded.columns)
for col in missing_cols:
  input_encoded[col]=0
input_encoded=input_encoded[X_encoded.columns]

#Predict pollutants
predicted_pollutants=model.predict(input_encoded)[0]

print(f"\nPredicted pollutant levels for station '{station_id}' in {year_input}:")
for p,val in zip(pollutants,predicted_pollutants):
  print(f"  {p}:  {val:2f}")

In [ ]:
import joblib

joblib.dump(model,'pollution_model.pkl')
joblib.dump(X_encoded.columns.tolist(),"model_columns.pkl")
print('Model and cols structure are saved')